# Generators

Generators allow you to declare a function that behaves like an iterator - a lazy iterator.
From https://realpython.com/introduction-to-python-generators/

Generators are for creating large lists that don't require large amount of memory like a list see PEP255. They behave like iterators but are implemented differently.

In [1]:
import random

# you could make generator create an infinite list
def random_number_sequence(limit):
    num = limit
    while num > 0:
        # use of 'yield' is what makes it a generator object rather than a standard object
        # state of generator function object saved here and number returned
        yield random.randint(0, 100)
        # start again from here when next() is called
        num -= 1

for i in random_number_sequence(10):
    print(i)

# the generator function is just a function
print("type(random_number_sequence) =", type(random_number_sequence))
# could be set to 100000000 without allocating equivalent memory
generator_fn = random_number_sequence(10)
print("..but object created when generator function is called is of type(generator_fn) =", type(generator_fn))

39
76
93
95
30
67
78
24
9
51
type(random_number_sequence) = <class 'function'>
..but object created when generator function is called is of type(generator_fn) = <class 'generator'>


The generator object is an iterator so we can call next() on a generator object...

In [2]:
print(next(generator_fn))
print(next(generator_fn))
print(next(generator_fn))

83
14
82


Using a generator expression or comprehension...

In [3]:
generator_comprehension = (random.randint(0, 100) for x in range(0, 10))
print("some random numbers")
for i in generator_comprehension:
    print(i)

my_string = 'hello'
print("converting the string '", my_string, "' to hexideximal")
hex_generator = ("{:02x}".format(ord(c)) for c in my_string)
print(type(hex_generator))
# join expects an iterable e.g. a list or a generator expression
print(":".join(hex_generator))

some random numbers
1
9
90
60
43
48
35
40
94
31
converting the string ' hello ' to hexideximal
<class 'generator'>
68:65:6c:6c:6f


Comparing a list comprehension with a generator comprehension...

In [4]:
nums_squared_lc = [num**2 for num in range(5)]
nums_squared_gc = (num**2 for num in range(5))
print("[num**2 for num in range(5)] =", nums_squared_lc)
print("(num**2 for num in range(5)) =", nums_squared_gc)

[num**2 for num in range(5)] = [0, 1, 4, 9, 16]
(num**2 for num in range(5)) = <generator object <genexpr> at 0x0000021157D9AFF0>


These do the same thing but have a different memory footprint. This creates a whole list through comprehension and then iterates.

In [5]:
for i in [num**2 for num in range(5)]:
    print(i)
for i in (num**2 for num in range(5)):          # this iterates data as it goes
    print(i)

0
1
4
9
16
0
1
4
9
16


`max()` also takes an iterable as a parameter so we can pass generator to max...

In [6]:
print(max(num**2 for num in range(5)))

16


What if i return a value from a generator?

In [7]:
def single_shot():
    yield 'fire!'
    return True             # returning value from generator function

my_single_shot = single_shot()
print(next(my_single_shot))
try:
    print(next(my_single_shot))
except StopIteration as e:              # StopIteration exception is raised
    print("StopIteration exception: generator finished and returned value:", e)

fire!
StopIteration exception: generator finished and returned value: True


Will a generator comprehension raise an exception?

In [8]:
single_shot_comprehension = (random.randint(0, 100) for x in range(0, 1))
print(next(single_shot_comprehension))
try:
    print(next(single_shot_comprehension))
except StopIteration as e:              # StopIteration exception is raised again
    # ...but no return value to print
    print("StopIteration exception: generator finished and returned value:", e)

57
StopIteration exception: generator finished and returned value: 


What attributes does a generator different from a standard function...

In [9]:
def single_shot_fn():
    fired = True
    return fired

print("difference in attributes between a generator function and standard function:",
      (set(dir(single_shot)) - set(dir(single_shot_fn))))
print("difference in attributes between a generator object and standard function:",
      (set(dir(my_single_shot)) - set(dir(single_shot_fn))))
print("difference in attributes between a generators from function or comprehension:",
      (set(dir(my_single_shot)) - set(dir(single_shot_comprehension))))

difference in attributes between a generator function and standard function: set()
difference in attributes between a generator object and standard function: {'gi_code', 'send', '__del__', '__next__', 'gi_frame', 'gi_yieldfrom', 'throw', 'close', 'gi_running', '__iter__'}
difference in attributes between a generators from function or comprehension: set()


You can have multiple yields...

In [10]:
def triple_shot():
    yield 'first yield!'
    yield 'second yield!'
    yield 'third yield!'

my_triple_generator = triple_shot()
print(next(my_triple_generator))
print(next(my_triple_generator))
print(next(my_triple_generator))

first yield!
second yield!
third yield!


How to use send() to create a coroutine...

In [11]:
def send_to_yield():
    # yield as expression and not statement
    ret = (yield 1)
    print("after first yield, ret=", ret)
    ret = (yield 2)
    print("after second yield, ret=", ret)
    yield                                           # yield None

try:
    print("\n")
    my_send_to_yield = send_to_yield()
    print("calling next(my_send_to_yield)")
    print("value returned from next =", next(my_send_to_yield)
          )                     # return first yielded value
    print("sending 'hello yield' to the generator")
    print("value returned from send =", my_send_to_yield.send(
        'hello yield'))       # send returns next yield value
    print("calling next(my_send_to_yield)")
    # returns last yield (None)
    print("value returned =", next(my_send_to_yield))
except StopIteration:
    print("generator complete!")



calling next(my_send_to_yield)
value returned from next = 1
sending 'hello yield' to the generator
after first yield, ret= hello yield
value returned from send = 2
calling next(my_send_to_yield)
after second yield, ret= None
value returned = None


Call throw on generator object...

In [12]:
def test_yield():
    try:
        yield 1
        yield 2
        yield 3
    except ValueError:
        print("caught value error exception in generator!")

my_throw_yield = test_yield()
value = next(my_throw_yield)
try:
    if value == 1:
        my_throw_yield.throw(ValueError("whoops don't like looks of this from generator"))
except StopIteration:
    print("caught stop iteration - thrown by throw() function")

caught value error exception in generator!
caught stop iteration - thrown by throw() function


Call close on generator object...

In [13]:
my_close_yield = test_yield()
value = next(my_close_yield)
if value == 1:
    my_close_yield.close()
try:
    # should be 2 if hadn't closed the generator
    value = next(my_close_yield)
except StopIteration:
    print("caught stop iteration - thrown by next() function")


caught stop iteration - thrown by next() function


`yield from` allows for delegation to sub-generators. See https://docs.python.org/3/whatsnew/3.3.html#pep-380.

In [14]:
my_sub_generator = ['bob', 'sally', 'donald']       # a list can be considered a sub-generator

def my_generator():
    yield from my_sub_generator                     # this could be any generator function or comprehension
    yield from range(3)
    yield from (x for x in my_sub_generator)
    yield (x for x in my_sub_generator)             # if I just yield, it will yield a generator object instead

for x in my_generator():
    print(x)

bob
sally
donald
0
1
2
bob
sally
donald
<generator object my_generator.<locals>.<genexpr> at 0x0000021157308200>


Can be useful for recursive traversal of tree...

In [15]:
my_nested_list = [[[1, 2, 3], 'bob', 'john'], ['sally', 'susan', 'amelia']]

def traverse_nested_lists(a_node):
    if type(a_node) is not list:
        print("yielding a node")
        yield a_node
    else:
        print("recurse nested list")
        for node in a_node:
            yield from traverse_nested_lists(node)

for x in traverse_nested_lists(my_nested_list):
    print(x)

recurse nested list
recurse nested list
recurse nested list
yielding a node
1
yielding a node
2
yielding a node
3
yielding a node
bob
yielding a node
john
recurse nested list
yielding a node
sally
yielding a node
susan
yielding a node
amelia
